In [4]:
# Basic libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from torchvision import datasets
from PIL import Image

In [5]:
# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# manual seed
torch.manual_seed(42)

In [7]:
# Greyscale to RGB
class ConvertToRGB:
    def __call__(self, image):
        return image.convert('RGB')
    

In [8]:
# Transformations
custom_transform = transforms.Compose([
    ConvertToRGB(),
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [9]:
# Fetch image folder
train_dir = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train'
dataset = datasets.ImageFolder(root = train_dir, transform = custom_transform)

In [10]:
# split dataset
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [11]:
# DataLoader class
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True, num_workers = 4, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size = 64, num_workers = 4, pin_memory = True)

In [12]:
# ResNet Model
import torchvision.models as models
ResNet = models.resnet34(pretrained = True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 220MB/s]


In [13]:
ResNet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
ResNet.fc

Linear(in_features=512, out_features=1000, bias=True)

In [15]:
# freeze layers
for param in ResNet.parameters():
    param.requires_grad = False

In [16]:
# custom classifer
ResNet.fc = nn.Sequential(
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(p = 0.5),
    
    nn.Linear(256, 1)
)

In [17]:
# unfreeze deeper layers
for param in ResNet.parameters():
    param.requires_grad = True

In [18]:
# define rate and epochs
learning_rate = 0.001
epochs = 10

In [19]:
# Model, loss and optimizer
ResNet = ResNet.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(ResNet.parameters(), lr = learning_rate, weight_decay = 1e-5)

In [20]:
# training loop
for epoch in range(epochs):
    total_epochs_loss = 0
    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device).float().unsqueeze(1)
        outputs = ResNet(batch_features)
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_epochs_loss += loss.item()
    avg_loss = total_epochs_loss / len(train_loader)
    print(f'Epoch: {epoch + 1}, Loss: {avg_loss}')

Epoch: 1, Loss: 0.22808761219792
Epoch: 2, Loss: 0.1691734072338565
Epoch: 3, Loss: 0.14940077409665806
Epoch: 4, Loss: 0.13739477259394672
Epoch: 5, Loss: 0.12472115063927455
Epoch: 6, Loss: 0.11347930343699057
Epoch: 7, Loss: 0.1040169392632392
Epoch: 8, Loss: 0.09334786025121114
Epoch: 9, Loss: 0.08411367141924309
Epoch: 10, Loss: 0.07595220193142578


In [21]:
# model eval
ResNet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
# evaluation
total = 0
correct = 0
for batch_features, batch_labels in test_loader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device).float().unsqueeze(1)
    outputs = ResNet(batch_features)
    probs = torch.sigmoid(outputs)
    preds = (probs > 0.5).float()
    total += batch_labels.shape[0]
    correct += (preds == batch_labels).sum().item()
acc = correct / total
print(f'Acc: {acc}')

Acc: 0.9607


In [24]:
# save the model
torch.save(ResNet.state_dict(), 'model_weights.pth')